> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Once you complete this project, remove these **Tip** sections from your report before submission. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Investigate a Dataset - tmdb_movies.csv

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected/downloaded for analysis. Read through the description available on the homepage-links present [here](https://docs.google.com/document/d/e/2PACX-1vTlVmknRRnfy_4eTrjw5hYGaiQim5ctr9naaRd4V9du2B5bxpd8FEH3KtDgp8qVekw7Cj1GLk1IXdZi/pub?embedded=True). List all column names in each table, and their significance. In case of multiple tables, describe the relationship between tables. 


### Question(s) for Analysis
>**Tip**: Clearly state one or more questions that you plan on exploring over the course of the report. You will address these questions in the **data analysis** and **conclusion** sections. Try to build your report around the analysis of at least one dependent variable and three independent variables. If you're not sure what questions to ask, then make sure you familiarize yourself with the dataset, its variables and the dataset context for ideas of what to explore.

> **Tip**: Once you start coding, use NumPy arrays, Pandas Series, and DataFrames where appropriate rather than Python lists and dictionaries. Also, **use good coding practices**, such as, define and use functions to avoid repetitive code. Use appropriate comments within the code cells, explanation in the mark-down cells, and meaningful variable names. 

In [1]:
# Use this cell to set up import statements for all of the packages that you
#   plan to use.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html
%matplotlib inline

In [ ]:
# Upgrade pandas to use dataframe.explode() function.
## removed ` =0.25.0` to get the latest version of pandas
##  because `df.head()` throws an error otherwise 
!pip install --upgrade pandas

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you **document your data cleaning steps in mark-down cells precisely and justify your cleaning decisions.**


### General Properties
> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

In [18]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
# pd.options.display.max_rows = 9999
df = pd.read_csv('Database_TMDb_movie_data/tmdb-movies.csv', sep=',')
# print(df.head)
df.head()
# type(df)

pandas.core.frame.DataFrame


### Data Cleaning
> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).
 

In [4]:
df.shape

(10866, 21)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
id                      10866 non-null int64
imdb_id                 10856 non-null object
popularity              10866 non-null float64
budget                  10866 non-null int64
revenue                 10866 non-null int64
original_title          10866 non-null object
cast                    10790 non-null object
homepage                2936 non-null object
director                10822 non-null object
tagline                 8042 non-null object
keywords                9373 non-null object
overview                10862 non-null object
runtime                 10866 non-null int64
genres                  10843 non-null object
production_companies    9836 non-null object
release_date            10866 non-null object
vote_count              10866 non-null int64
vote_average            10866 non-null float64
release_year            10866 non-null int64
budget_adj              1

In [9]:
df.duplicated().value_counts()

False    10865
True         1
dtype: int64

# Structure
* 10866 observations
* 21 variables:
    * 11 are object
    * 6 are int
    * 4 are float
* Two unique IDs for each observation:
    * `id`
    * `imdb_id`
    * any one will suffice in this analysis
* `budget` and `revenue`:
    * `budget` and `revenue` are in units of money in the year of production
    * `budget_adj` and `revenue_adj` are, simply, real (inflation-adjusted) `budget` and `revenue` respectively
* `release_date` is in ***M/D/YY*** format

# Missing and null values
* Observations have missing values
* Case(s) in point
    * 7930 missing values in the `homepage` variable
    * 76 missing values in the `cast` variable
    
# Multiple values for a variable
* Variables `cast`, `genre` and `production_companies` use the pipe symbol, `|`, as a seperator

# Duplicate observations
* 1 duplicate observation
    * will investigate and drop if appropriate 

# Cleaning

* Drop `homepage` and `imdb_id`
    * `homepage` contains URLs; these will not be used in this analysis
    * `imdb_id` and `id` serve the same purpose in this analysis. `id` suffices
            
* See if the the number of missing values of the remaining variables is significant enough to drop
    * find the number of missing values per variable
    * find the proportion of said missing values per variable  to the total
    * if proportion exceeds a threshold, fill missing with median, else, drop observation

* Make data types of variables consistent
    * change the following to `int`:
         * `revenue_adj`
         * `budget_adj`
         * `popularity`
         * `vote_average`

In [10]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.
df.drop(['homepage', 'imdb_id'], axis=1, inplace=True)

In [11]:
#list of columns
variables = list(df.columns)

In [12]:
#list containing the number of missing values per variable
num_missing = list(df.isnull().sum())

In [13]:
#use a dict comp to see `variables` and `num_missing` together
missing_values = {k:v for (k,v) in zip(variables, num_missing)}

In [14]:
#list containing proportion of missing values to total on a per variable basis
# 4 dp, percent
missing_proportion = [f'{i/10866:0.4f}%' for i in num_missing]

In [15]:
#see the proportions per variable
missing_val_proportions = {k:v for (k,v) in zip(variables, missing_proportion)}


## An overwhelming majority of variables have little to no missing values

### To fill
* All variables:
    * with missing values
    * whose proportion of missing values is 0.0000%
    * whose values are not categorical

### To drop
* `cast`. I can easily find these online, however, that is outside the scope of this project
* `director`. I can easily find these online, however, that is outside the scope of this project
* `tagline`. The proportion is too high to fill meaningfully because `tagline` is a [qualitative variable](https://www.statisticshowto.com/qualitative-variable/)
* `keywords`. I can easily find these online, however, that is outside the scope of this project
* `overview`. I can easily find these online, however, that is outside the scope of this project
* `genres`. I can easily find these online, however, that is outside the scope of this project
* `production_companies`. I can easily find these online, however, that is outside the scope of this project


In [16]:
#drop values from variables discussed above
to_drop_list = [i for i in variables if missing_val_proportions[i] != '0.0000%']
df.dropna(axis='index', subset=to_drop_list, inplace=True)
df.info() #7032 obs, 19 variables

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 10865
Data columns (total 19 columns):
id                      7032 non-null int64
popularity              7032 non-null float64
budget                  7032 non-null int64
revenue                 7032 non-null int64
original_title          7032 non-null object
cast                    7032 non-null object
director                7032 non-null object
tagline                 7032 non-null object
keywords                7032 non-null object
overview                7032 non-null object
runtime                 7032 non-null int64
genres                  7032 non-null object
production_companies    7032 non-null object
release_date            7032 non-null object
vote_count              7032 non-null int64
vote_average            7032 non-null float64
release_year            7032 non-null int64
budget_adj              7032 non-null float64
revenue_adj             7032 non-null float64
dtypes: float64(4), int64(6), object(9)

In [19]:
'''
Function to drop observations in data set

Takes in 1 arg: the data set

Data set must be a pandas dataframe

returns a clean dataset if successful, else, none
'''

def drop_rows(df):
    """function drop_rows"""
    df = df if df and type(df) == pandas.core.frame.DataFrame else None
    variables = list(df.columns)
    if df:
        try:
            try:
                df.drop(['homepage', 'imdb_id'], axis=1, inplace=True)
            except:
                print(f'Failed to drop rows associated with selected columns')
            try:
                num_missing = list(df.isnull().sum())
                missing_values = {k:v for (k,v) in zip(variables, num_missing)}           
                missing_proportion = [f'{i/10866:0.4f}%' for i in num_missing]
                missing_val_proportions = {k:v for (k,v) in zip(variables, missing_proportion)}
                to_drop_list = [i for i in variables if missing_val_proportions[i] != '0.0000%']
                df.dropna(axis='index', subset=to_drop_list, inplace=True)
                df.info()
            except:
                print(f'Failed to drop null values')
        except:
            print(f'Drop operation failed')

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. **Compute statistics** and **create visualizations** with the goal of addressing the research questions that you posed in the Introduction section. You should compute the relevant statistics throughout the analysis when an inference is made about the data. Note that at least two or more kinds of plots should be created as part of the exploration, and you must  compare and show trends in the varied visualizations. 



> **Tip**: - Investigate the stated question(s) from multiple angles. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables. You should explore at least three variables in relation to the primary question. This can be an exploratory relationship between three variables of interest, or looking at how two independent variables relate to a single dependent variable of interest. Lastly, you  should perform both single-variable (1d) and multiple-variable (2d) explorations.




### Research Question 1

* Which combination of genres are most popular?

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


In [34]:
popular = df.groupby('genres')['revenue'].sum()
popular

genres
Action                                                 2331375459
Action|Adventure                                        484253103
Action|Adventure|Animation                                      0
Action|Adventure|Animation|Comedy|Drama                  16627188
Action|Adventure|Animation|Comedy|Family                        0
Action|Adventure|Animation|Drama|Family                         0
Action|Adventure|Animation|Family                               0
Action|Adventure|Animation|Family|Fantasy               554987477
Action|Adventure|Animation|Family|Mystery                       0
Action|Adventure|Animation|Family|Science Fiction       631442092
Action|Adventure|Animation|Fantasy                              0
Action|Adventure|Animation|Fantasy|Horror                       0
Action|Adventure|Animation|Fantasy|Science Fiction              0
Action|Adventure|Animation|Science Fiction                5231128
Action|Adventure|Animation|Science Fiction|Crime                0
Act

### Research Question 2 
* What properties are associated with movies that have high revenue?
    * Does the season of the year in which a movie is released affect revenue?
    * How does popularity correlate with revenue?
    * Do movies from a particular director perform better in revenue terms?
    * Is there a year that is particularly good from a revenue perspective?

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed in relation to the question(s) provided at the beginning of the analysis. Summarize the results accurately, and point out where additional research can be done or where additional information could be useful.

> **Tip**: Make sure that you are clear with regards to the limitations of your exploration. You should have at least 1 limitation explained clearly. 

> **Tip**: If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> **Tip**: Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> **Tip**: Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> **Tip**: Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])